In [1]:
from plotly.graph_objs import *
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import cmocean
import numpy as np
import xarray as xr
import plotly
from scipy.ndimage import gaussian_filter
from metpy.units import units
#https://plot.ly/python/cmocean-colorscales/
#https://community.plot.ly/t/subloting-mapbox-in-jupyter-notebook/16126/6
# color 的设置 https://stackoverflow.com/questions/43839629/python-plotly-create-a-color-scale-related-to-max-and-min-number-of-value
# https://plot.ly/python/colorscales/
# https://plot.ly/python/scattermapbox/

C:\Users\xigrug\Anaconda3\Anaconda\lib\site-packages\dask\config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

C:\Users\xigrug\Anaconda3\Anaconda\lib\site-packages\distributed\config.py:20: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [2]:
# mapbox_access_token = 'ADD_YOUR_TOKEN_HERE'
mapbox_access_token = 'pk.eyJ1IjoieGlncnVnIiwiYSI6ImNqZm8ycXMxejAwOXYyenM3Z2twOWJvb2EifQ.lSikTXoOmtTjHutewtCe9A'
#mapbox_access_token = 'pk.eyJ1Ijoiam1tZWFzZSIsImEiOiJjamljeWkwN3IwNjEyM3FtYTNweXV4YmV0In0.2zbgGCjbPTK7CToIg81kMw'
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [3]:
def cmocean_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    newcmap = cmocean.tools.crop_by_percent(cmap, 30, which='both', N=None)
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        # https://stackoverflow.com/questions/6800481/python-map-object-is-not-subscriptable
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

In [5]:
#cmap = cmocean.cm.matter
cmap = cmocean.cm.thermal
newcmap = cmocean.tools.crop_by_percent(cmap, 30, which='both', N=None)
curl = cmocean_to_plotly(newcmap, 10)

In [20]:
du = xr.open_dataset('C:/Users/xigrug/Desktop/CCN/ERA5_u-2013-2018_6-8_0500.nc')
dv = xr.open_dataset('C:/Users/xigrug/Desktop/CCN/ERA5_v-2013-2018_6-8_0500.nc')

In [21]:
lat1=du.latitude
lon1=du.longitude
u=du.u.data
v=dv.v.data
uwnd_850 = gaussian_filter(u, sigma=3.0) * units('m/s')
vwnd_850 = gaussian_filter(v, sigma=3.0) * units('m/s')

In [6]:
df=pd.read_table("AUTO_5MINS_NPP_MicroPhys_20130627_0521_filter.csv",sep=',',na_values=[-9999])
print(df.head())

#df['text'] = df['latbin'] + '' + df['lonbin'] + ', ' + df['CCN_1.0_cm3'] +','+ df['SS_1.0_mg']
df=df[df["CCN_1.0_cm3"]<10000]
df=df[df["CCN_1.0_cm3"]>0]
site_lat = df.lat
site_lon = df.lon
locations_name = df['CCN_1.0_cm3']

l0_name = df['SS_1.0_mg']

   winNO  big_c  big_r      lat       lon  colum   row    Have    Hsd    Hmax  \
0    703   4384   6112  23.1861  121.0311   4192  6112  1668.0  791.0  3416.0   
1    705   4384   6112  23.5610  121.4422   4288  6016   584.0  796.0  3076.0   
2    706   4384   6112  23.4993  120.9580   4192  6016  2128.0  630.0  3661.0   
3    708   4384   6112  23.8739  121.3693   4288  5920  1300.0  926.0  3075.0   
4    709   4384   6112  23.8136  120.8951   4192  5920   907.0  586.0  3064.0   

   ...  AMR_surf  SMR_surf  T_LCL_surf  H_LCL_lowest  AMR_lowest  SMR_lowest  \
0  ...       NaN       NaN         NaN           NaN         NaN         NaN   
1  ...       NaN       NaN         NaN           NaN         NaN         NaN   
2  ...       NaN       NaN         NaN           NaN         NaN         NaN   
3  ...       NaN       NaN         NaN           NaN         NaN         NaN   
4  ...       NaN       NaN         NaN           NaN         NaN         NaN   

   T_LCL_lowest  CAPE_surf  CAPE

C:\Users\xigrug\Anaconda3\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

read_table is deprecated, use read_csv instead.



In [15]:
locations_name1=pd.cut(locations_name,10)

In [20]:
locations_name2=locations_name1[0]

In [23]:
locations_name2

Interval(55.093, 550.7, closed='right')

In [7]:
print(df['CCN_1.0_cm3'].max())

4967.0


In [17]:
data = Data([
    go.Scattermapbox(
        # 绘制散点的经纬度
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=8,
            color=locations_name1,
            showscale=True,
            cmax=5000,
            cmin=200,
            colorscale=curl
        ),
        # 散点对应的文本
        text=locations_name,
        customdata=l0_name,
        name='CCN'
    )
])

ValueError: 
    Invalid element(s) received for the 'color' property of scattermapbox.marker
        Invalid elements include: [Interval(55.093, 550.7, closed='right'), Interval(55.093, 550.7, closed='right'), Interval(1041.4, 1532.1, closed='right'), Interval(550.7, 1041.4, closed='right'), Interval(1532.1, 2022.8, closed='right'), Interval(550.7, 1041.4, closed='right'), Interval(550.7, 1041.4, closed='right'), Interval(550.7, 1041.4, closed='right'), Interval(550.7, 1041.4, closed='right'), Interval(55.093, 550.7, closed='right')]

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
       

In [9]:
layout = Layout(
    #title='CCN 2013-06-28_05:03_filter',
    title='CCN 2013-06-27_05:21_filter',
    autosize=True,
    width=800,
    height=800,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        domain={'x': [0, 1.0], 'y': [0, 1]},
        bearing=0,
        # 地图中心坐标，不要远离绘制的散点坐标
        center=dict(
            lat=32,
            lon=118
        ),
        pitch=0,
        zoom=4
    ),
)

fig = go.FigureWidget(data=data, layout=layout)
fig

In [10]:
fig = go.Figure(data=data, layout=layout)

In [11]:
plotly.offline.iplot(fig)
plotly.offline.plot(fig, filename="ccn_20130627_0521_filternew.html", auto_open=False)

'ccn_20130627_0521_filternew.html'